# lesson 5

1、把上一lesson中得到的蛋白序列fasta文件(如名字叫Cflo.pep)，作为输入文件，编写一个脚本，把Cflo.pep切割成100个子文件，每个文件的序列条数尽量相等，然后输出到一个叫split的目录下，子文件名可以是Cflo.pep.1、Cflo.pep.2、……、Cflo.pep.100。该perl脚本需要会自己判断split/目录是否已经存在，如果不存在则自动创建。

2、编写一个脚本(script1.pl)，用于判断一条蛋白序列的第一个氨基酸是否是M，如果一条蛋白序列的第一个氨基酸是M，就输出这条蛋白。编写第二个脚本(script2.pl)，用于自动获取split/目录下的100个子文件，然后对每一个子文件生成一条执行第一个脚本的命令行，并把命令行(如perl script1.pl Cflo.pep.1 >Cflo.pep.1.out)输出到一个shell文件中(如Cflo.pep.1.sh)，从而得到100个子文件对应的100个shell文件。最后把这100个shell文件用qsub提交到计算节点中并行执行。


In [23]:
import os
if os.path.exists('./split/'):
    path = './split/'
else:
    os.mkdir('split')
    path = './split/'
    
pep_num = 0
with open('./lesson4_protein.fa','r') as f:
    for line in f:
        if line.startswith('>'):
            pep_num += 1
num_perfile = [pep_num // 100] * 100
for i, j in enumerate(num_perfile):
    if i < pep_num % 100 :
        num_perfile[i] += 1
        
f = open('./lesson4_protein.fa', 'r')
count = 0
file_num = 1
limit_num = num_perfile.pop()
w = open(path + 'Cflo.pep_'+str(file_num), 'w')
for line in f:
    if line.startswith('>'):
        count += 1         
    if count <= limit_num:
        w.write(line)
    else:
        w.close()
        limit_num = num_perfile.pop()
        file_num += 1
        count = 1
        w = open(path + 'Cflo.pep_'+str(file_num), 'w')
        w.write(line)
w.close()
                

上述脚本即可将lesson4中的蛋白序列文件lesson4_protein.fa切割为100个文件Cflo.pep_1、Cflo.pep_2、...、Cflo.pep_100。蛋白序列总条数为17064，切割后每个文件蛋白序列为170条或171条。所有文件都在./split/文件夹下。

### 判断蛋白序列起始氨基酸

首先编写脚本script1.py用于判断一条蛋白序列的第一个氨基酸是否为M。若以M开头，print这条序列。

In [ ]:
import sys

filename = sys.argv[1]         #命令行参数
f = open(filename, 'r')
pep = []
pro_name = ''
pro_seq = []
for line in f:
    if line.startswith('>'):
        if pro_name:
            seq = ''.join(pro_seq)
            if seq.startswith('M'):          #判断序列是否以M起始。
                pep.append([pro_name, seq])
            pro_name = line
            pro_seq = []
        else:
            pro_name = line
    else:
        pro_seq.append(line)
seq = ''.join(pro_seq)
if seq.startswith('M'):          
    pep.append([pro_name, seq])  
f.close()

all_seq = ''.join([n + s for n, s in pep])
print(all_seq)

上面的脚本script1.py可以将存储蛋白序列的fasta文件中以M起始的蛋白序列提取出来，并打印到标准输出。

`python script1.py Cflo.pep_1 > out_1`

即可提取Cflo.pep_1中以M开头的蛋白序列并输入至out_1中。结果部分截图：

![out_1](./image/lesson5_M.JPG)

<hr>
接下来编写脚本script2.py。用于自动获取split/目录下的100个子文件，然后对每一个子文件生成一条执行script1.py的命令行，并把命令行输出到一个shell文件中。

In [ ]:
import os
files = os.listdir('./split')
os.mkdir('shell_script')
path = 'shell_script/'
for i in files:
    w = open(path + i + '.sh', 'w')
    w.write('python script1.py {} > out_{}'.format(i, i.split('_')[-1]))
    w.close()

运行script2就可以生成100个对应的sh文件，储存在./shell_script文件夹下。如下图：

![shell_script](./image/lesson5_sh.JPG)

每个sh文件内容形如：

`python script1.py Cflo.pep_1 > out_1`

## Reference

[Python3.6 os module](https://docs.python.org/3.6/library/os.html#module-os)

[Python 脚本传参方式](https://blog.csdn.net/weixin_35653315/article/details/72886718)